In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

df = pd.read_csv('WineQuality.csv')
df.dropna(inplace = True)

X = df.drop(['quality'], axis = 1)
y = df.quality
X = MinMaxScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 4, stratify = y)
logreg = LogisticRegression(random_state = 101, class_weight = 'balanced')
logreg.fit(X_train, y_train)
logreg_pred = logreg.predict(X_test)
lr_acc = metrics.accuracy_score(y_test, logreg_pred)

rf = RandomForestClassifier(oob_score = True, class_weight = 'balanced', random_state = 2)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
rf_acc = metrics.accuracy_score(y_test,y_pred)

xgb_model = XGBClassifier(num_class = 7, objective = "multi:softprob", eval_metric = "mlogloss", seed = 42, random_state = 7)
xgb_model.fit(X_train, y_train)
xgboost_pred = xgb_model.predict(X_test)
xgb_acc = metrics.accuracy_score(y_test, xgboost_pred)

with open("output.csv", mode = "w") as f:
    f.write(str(lr_acc))
    f.write('\n')
    f.write(str(rf_acc))
    f.write('\n')
    f.write(str(xgb_acc))

D:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
D:\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [46]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv('WineQuality.csv')
df.dropna(inplace = True)

X = df.drop(['quality'], axis = 1)
y = df.quality
X = MinMaxScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1234, stratify = y)
null_accuracy = y_test.value_counts().head(1) / len(y_test)
null_acc = null_accuracy.get(key = 6.0)

with open("output.csv", mode = "w") as f:
    f.write(str(null_acc))

In [65]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score

df = pd.read_csv('Bank.csv')
cols = df.describe(include = 'O').columns
df[cols] = df[cols].apply(LabelEncoder().fit_transform)

X = df.drop(['y'], axis = 1)
y = df.y
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 2, test_size = 0.25)

rf = RandomForestClassifier(criterion = 'entropy', oob_score = True, class_weight = 'balanced', random_state = 2)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
rf_acc = round(metrics.accuracy_score(y_test,y_pred), 3)

predict_probabilities = rf.predict_proba(X_test)
final_metric = round(roc_auc_score(y_test, predict_probabilities[:,1]), 3)

rf_gs = GridSearchCV(estimator = rf, param_grid = {'n_estimators':[80,90,100]}, cv = 2)
rf_gs.fit(X_train, y_train)
y_pred_gs = rf_gs.predict(X_test)
rf_acc_gs = round(metrics.accuracy_score(y_test,y_pred), 3)

predict_probabilities = rf_gs.predict_proba(X_test)
final_metric_gs = round(roc_auc_score(y_test, predict_probabilities[:,1]), 3)

with open("output.csv", mode = "w") as f:
    f.write(str(rf_acc))
    f.write('\n')
    f.write(str(final_metric))
    f.write('\n')
    f.write(str(rf_acc_gs))
    f.write('\n')
    f.write(str(final_metric_gs))

In [75]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

df = pd.read_csv('Bank.csv')
cols = df.describe(include = 'O').columns
df[cols] = df[cols].apply(LabelEncoder().fit_transform)

X = df.drop(['y'], axis = 1)
y = df.y
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 3, test_size = 0.3)

lr = LogisticRegression(class_weight = 'balanced')
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_acc = round(metrics.accuracy_score(y_test, lr_pred), 3)

predict_probabilities = lr.predict_proba(X_test)
final_metric = round(metrics.roc_auc_score(y_test, predict_probabilities[:,1]), 3)

if (lr_acc < final_metric):
    var = 'ROC-AUC'
else:
    var = 'Accuracy'

with open("output.csv", mode = "w") as f:
    f.write(str(lr_acc))
    f.write('\n')
    f.write(str(final_metric))
    f.write('\n')
    f.write('\n')
    f.write(str(var))

D:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
